In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'f:\\Personal Project\\Medical-Image-Segmentation'

In [ ]:
import dagshub
dagshub.init(repo_owner='TuanMinhajSeedin', repo_name='Medical-Image-Segmentation', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [2]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/TuanMinhajSeedin/Medical-Image-Segmentation.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="TuanMinhajSeedin"
os.environ["MLFLOW_TRACKING_PASSWORD"]="02e67d288aa3d8ba149140060424f74a41d96d0c"

In [ ]:
export MLFLOW_TRACKING_URI=os.environ["MLFLOW_TRACKING_URI"]=https://dagshub.com/TuanMinhajSeedin/Medical-Image-Segmentation.mlflow
export MLFLOW_TRACKING_USERNAME=TuanMinhajSeedin
export MLFLOW_TRACKING_PASSWORD=02e67d288aa3d8ba149140060424f74a41d96d0c

In [7]:
import tensorflow as tf


In [15]:
model_path = "artifacts/training/model.h5"
model = tf.keras.models.load_model(
            model_path,
            compile=False
            )

In [16]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [18]:
from LiverTumorSegmentation.constants import *
from LiverTumorSegmentation.utils.common import read_yaml,create_directories,save_json

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=self.config.trained_model_path,
            training_data=self.config.training_data,
            mlflow_uri=self.config.mlflow_uri,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [19]:
import tensorflow as tf
import pathlib as Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [20]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        # self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e